In [80]:
import pymatgen as pmg
from pymatgen.io.vasp.outputs import Outcar
from pymatgen.electronic_structure.core import Spin
import numpy as np

from os.path import expanduser
home = expanduser('~')

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


def display_occu_sep_spin(odm):
    up = odm[Spin.up]
    down = odm[Spin.down]
    
    for i, matrix in enumerate([up, down]):
        if i == 0: print('UP')
        else: print('\n\nDOWN')
        header = '\t'
        for orbital in orbitals:
            header = f'{header}{orbital}\t'
        print(header, end='')
        for j, orbital in enumerate(orbitals):
            print(f'\n{orbital}', end='\t')
            for k in range(len(orbitals)):
                print(f'{matrix[j][k]}', end='\t')
    print('\n')
    
def display_occu(odm):
    up = odm[Spin.up]
    down = odm[Spin.down]
    header = '\t'
    for orbital in orbitals:
        header = f'{header}{orbital}\t\t'
    print(header, end='')
    for j, orbital in enumerate(orbitals):
        print(f'\n{orbital}', end='\t')
        for k in range(len(orbitals)):
            entries = [np.round(up[j][k], 1), np.round(down[j][k], 1)]
            for i in range(2):
                if entries[i] == 0:
                    entries[i] = '---'
            print(f'{entries[0]} / {entries[1]}', end='\t')
    print('\n')

In [81]:
PREFIX = home + '/Projects/BaCoS2/correct_mag/estruc/OUTCAR'
SUFFIX = ""
ITERS_1 = ['U0', 'exp-U0', 'U2', 'exp-U2',  'U2.2', 'U4', 'exp-U4']

LABELS_1 = ITERS_1

orbitals_fancy = ['$d_{x^2 - y^2}$', '$d_{yz}$', '$d_{z^2 - r^2}$', '$d_{xz}$', '$d_{xy}$']
orbitals = ['d_x2y2', 'd_yz', 'd_z2-r2', 'd_xz', 'd_xy']

odms = []

for i, iter_1 in enumerate(ITERS_1):
        label = f'{iter_1}'
        outcar = Outcar(f'{PREFIX}-{label}{SUFFIX}')
        outcar.read_onsite_density_matrices()
        odm = outcar.data['onsite_density_matrices']
        odms.append(odm)
        print(f'##### {iter_1} #####')
        display_occu(odm[0])

##### U0 #####
	d_x2y2		d_yz		d_z2-r2		d_xz		d_xy		
d_x2y2	1.0 / 0.5	--- / ---	--- / ---	--- / ---	--- / ---	
d_yz	--- / ---	1.0 / 0.5	--- / ---	--- / ---	--- / ---	
d_z2-r2	--- / ---	--- / ---	1.0 / 0.5	--- / ---	--- / ---	
d_xz	--- / ---	--- / ---	--- / ---	1.0 / 0.5	--- / ---	
d_xy	--- / ---	--- / ---	--- / ---	--- / ---	1.0 / 0.5	

##### exp-U0 #####
	d_x2y2		d_yz		d_z2-r2		d_xz		d_xy		
d_x2y2	1.0 / 0.5	--- / ---	--- / ---	--- / ---	--- / ---	
d_yz	--- / ---	1.0 / 0.5	--- / ---	--- / ---	--- / ---	
d_z2-r2	--- / ---	--- / ---	1.0 / 0.5	--- / ---	--- / ---	
d_xz	--- / ---	--- / ---	--- / ---	1.0 / 0.5	--- / ---	
d_xy	--- / ---	--- / ---	--- / ---	--- / ---	1.0 / 0.5	

##### U2 #####
	d_x2y2		d_yz		d_z2-r2		d_xz		d_xy		
d_x2y2	1.0 / 0.5	--- / ---	--- / ---	--- / ---	--- / ---	
d_yz	--- / ---	1.0 / 0.5	--- / ---	--- / ---	--- / ---	
d_z2-r2	--- / ---	--- / ---	1.0 / 0.5	--- / ---	--- / ---	
d_xz	--- / ---	--- / ---	--- / ---	1.0 / 0.5	--- / ---	
d_xy	--- / ---	--- / ---	--- / ---	--- 